# <strong>3b: Apex Only Notebook |</strong>


In [1]:
import os
import glob
import pandas as pd
from scipy.signal import find_peaks
import numpy as np
import warnings
warnings.simplefilter('ignore', category=FutureWarning)

# Function to align data

This Python function, `align_data`, is designed to align two dataframes: `mtdf` and `annotdf` which will contain the motiontracking data and the ELAN annotations.


In [ ]:
# Function to align data
def align_data(mtdf, annotdf):
# Add a gesture ID column to the annotations dataframe
    annotdf['gesture_id'] = range(1, len(annotdf) + 1)

    # Create new columns in data_df for annotations and gesture IDs
    mtdf['phase'] = None
    mtdf['gesture_id'] = None

    # Iterate over each row in the annotations dataframe
    for _, row in annotdf.iterrows():
        # Find the range of time in data_df that falls within the current annotation time
        condition = (mtdf['time_ms'] >= row['start_time']) & (mtdf['time_ms'] <= row['end_time'])
        
        # Assign the annotation and gesture ID to these rows
        mtdf.loc[condition, 'phase'] = row['phase']
        mtdf.loc[condition, 'gesture_id'] = row['gesture_id']

    return mtdf

In [ ]:
# Define directories
phase_dir = '/n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/elan_phases/SN6014_ARTGEST_ENG'
processed_dir = '/n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/speed_and_upsample/SN6014_ARTGEST_ENG'
output_dir = '/n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/apex_annotated/SN6014_ARTGEST_ENG'

# Output Structuring
output_file = os.path.join(output_dir, os.path.basename(processed_file).replace('_processed_data.csv', '_final_annotations.csv'))

# Get list of files in directories
phase_files = glob.glob(os.path.join(phase_dir, '*.csv'))
processed_files = glob.glob(os.path.join(processed_dir, '*.csv'))

# List gathered files
phase_files
processed_files

In [ ]:
# Match phase and processed files based on filenames
matched_files = []
for phase_file in phase_files:
    basename = os.path.basename(phase_file).split('_extended_annotations.csv')[0]
    for processed_file in processed_files:
        if basename in processed_file:
            matched_files.append((phase_file, processed_file))
            break

In [ ]:
# Process each matched file pair
for phase_file, processed_file in matched_files:
    df_phase = pd.read_csv(phase_file)
    df_processed = pd.read_csv(processed_file)

    # Rename columns if necessary
    df_phase.columns = ['start_time', 'end_time', 'phase']

    # Main processing logic
    df_processed = align_data(df_processed, df_phase)
    df_processed = df_processed.drop(columns=['right_wrist_delta_x', 'right_wrist_delta_y', 'right_index_x',
                                              'right_index_y', 'right_index_speed', 'right_index_delta_x','right_index_delta_y'])
    filtered_df = df_processed.dropna(subset=['right_wrist_speed', 'phase'])
    more_refined_apexes_df = pd.DataFrame(columns=['time_ms', 'right_wrist_speed', 'gesture_id', 'apex'])

    for gesture_id in filtered_df['gesture_id'].dropna().unique():
        gesture_data = filtered_df[filtered_df['gesture_id'] == gesture_id]
        stroke_data = gesture_data[gesture_data['phase'].isin(['S', 'Stroke'])]
        if len(stroke_data) == 0:
            continue
        dynamic_large_peak_threshold = stroke_data['right_wrist_speed'].mean() + stroke_data['right_wrist_speed'].std()
        peaks, _ = find_peaks(stroke_data['right_wrist_speed'].values, height=dynamic_large_peak_threshold)
        if len(peaks) > 0:
            for peak in peaks:
                subsequent_data = stroke_data['right_wrist_speed'].values[peak:]
                if len(subsequent_data) > 1:
                    min_speed_index = np.argmin(subsequent_data) + peak
                    original_index = stroke_data.index[min_speed_index]
                    stroke_data.loc[original_index, 'apex'] = 'AX'
        else:
            min_speed_index = np.argmin(stroke_data['right_wrist_speed'].values)
            original_index = stroke_data.index[min_speed_index]
            stroke_data.loc[original_index, 'apex'] = 'AX'
        more_refined_apexes_df = more_refined_apexes_df._append(stroke_data)

    # Save output
    more_refined_apexes_df.to_csv(output_file, index=False)

    print(f'Done Processing {output_file}')